In [2]:
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import utils, datasets
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler

from sklearn.metrics import classification_report, confusion_matrix

from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization, training

In [4]:
model = InceptionResnetV1(num_classes=3)
device = torch.device("cuda")
model.to(device)

InceptionResnetV1(
  (conv2d_1a): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2a): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2b): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (maxpool_3a): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2d_3b): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_4a): 

In [20]:
train_data_dir = '../../../../datasets/MaskedFace-Net/train'
test_data_dir = '../../../../datasets/MaskedFace-Net/validation'

batch_n = 32 
epochs = 3 

In [6]:
img_transforms = {
    'train': T.Compose([T.Resize((256,256)), T.ToTensor()]),
    'val' : T.Compose([T.Resize((256,256)), T.ToTensor()])
}

In [22]:
mask_data = datasets.ImageFolder(root= train_data_dir, transform = img_transforms['train'])
test_mask_data = datasets.ImageFolder(root = test_data_dir, transform=img_transforms['val'])

In [8]:
mask_data.class_to_idx

{'covered': 0, 'incorrect': 1, 'uncovered': 2}

In [9]:
n = len(mask_data)
indices = list(range(n))
np.random.shuffle(indices)
indices

[8296,
 13035,
 7489,
 29817,
 10654,
 23924,
 32650,
 12785,
 17902,
 6725,
 3751,
 1239,
 19012,
 25047,
 33275,
 29675,
 11587,
 622,
 22963,
 5834,
 23645,
 28623,
 30492,
 35033,
 26812,
 8708,
 28699,
 17005,
 7991,
 20627,
 24032,
 1559,
 16023,
 1116,
 7425,
 5172,
 5828,
 29974,
 32448,
 8233,
 29092,
 26818,
 17759,
 32008,
 35500,
 24154,
 28116,
 1431,
 16315,
 22011,
 35519,
 9147,
 15488,
 25928,
 35308,
 31051,
 21732,
 35550,
 20065,
 872,
 16963,
 7070,
 20844,
 29089,
 31014,
 29444,
 31699,
 29212,
 18801,
 25414,
 14558,
 30463,
 27667,
 15900,
 15338,
 33721,
 27604,
 10676,
 22066,
 36513,
 6163,
 27374,
 29326,
 26462,
 22959,
 32833,
 5641,
 6171,
 20470,
 16363,
 18760,
 11059,
 24951,
 16232,
 1406,
 23608,
 3175,
 34971,
 11936,
 33666,
 29213,
 32104,
 25361,
 11946,
 27048,
 17497,
 24656,
 1580,
 10461,
 21166,
 8810,
 23601,
 36457,
 20597,
 11970,
 16940,
 24377,
 12811,
 33847,
 32221,
 9847,
 4599,
 27462,
 14216,
 21141,
 2696,
 7746,
 35241,
 26822,


In [10]:
train_sampler = SubsetRandomSampler(indices[:8000])
train_loader = DataLoader(dataset=mask_data, shuffle=False, batch_size = 8, sampler=train_sampler)

In [37]:
test_indices = list(range(len(test_mask_data)))
np.random.shuffle(test_indices)
test_sampler =SubsetRandomSampler(test_indices[:3000])
test_loader = DataLoader(dataset=test_mask_data, shuffle=False, batch_size = 8, sampler=test_sampler)

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.009)

In [15]:
def cross_accuracy(y_pred,y): 
    ypt = torch.log_softmax(y_pred, dim=1)
    temp, ypt = torch.max(ypt, dim=1)
    acc = ((ypt == y).sum() / len(y)) * 100
    return acc

In [16]:
accuracy_stats = {
    'train': [],
    "val": []
}
loss_stats = {
    'train': [],
    "val": []
}

In [18]:
print('Begin Training') 

for e in tqdm(range(0,2)):
    e_loss, e_acc = 0, 0
    model.train()
    for X_train, y_train in train_loader: 
        X_train, y_train = X_train.to(device), y_train.to(device)
        optimizer.zero_grad()
        ytp = model(X_train).squeeze()
        t_loss = criterion(ytp, y_train)
        t_acc = cross_accuracy(ytp, y_train)
        t_loss.backward()
        optimizer.step()
        
        e_loss += t_loss.item()
        e_acc += t_acc.item()
    loss_stats['train'].append(e_loss/len(train_loader))
    accuracy_stats['train'].append(e_acc/len(train_loader))
    print("Done with Epoch {}".format(e))

print('Training Finished')

Begin Training


Done with Epoch 0
Done with Epoch 1

Training Finished


In [19]:
accuracy_stats

{'train': [72.1, 77.7], 'val': []}

In [38]:
y_preds = []
y_true = []
with torch.no_grad(): 
    for x_test, y_test in tqdm(test_loader): 
        x_test, y_test = x_test.to(device), y_test.to(device)
        ytp = model(x_test)
        temp, ypt = torch.max(ytp, dim=1)
        y_preds.append(ypt.cpu().numpy())
        y_true.append(y_test.cpu().numpy())

In [48]:
y_preds_flat = np.array(y_preds).flatten()

In [47]:
y_true_flat = np.array(y_true).flatten()

In [50]:
print(classification_report(y_true_flat, y_preds_flat))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95       971
           1       0.96      0.95      0.96      1020
           2       0.98      0.99      0.98      1009

    accuracy                           0.96      3000
   macro avg       0.96      0.96      0.96      3000
weighted avg       0.96      0.96      0.96      3000



## Save model to pickle

In [53]:
torch.save(model, '../Config/inceptionResnetV1.pth')

In [56]:
### if you want to load the model, run the below commented code

# model_path = '../Config/inceptionResnetV1.pth'
# model = torch.load('../Config/inceptionResnetV1.pth')
# model.eval()